#Boston 311 Tutorial

This notebook will run you through the basic usage of this package to train 3 models on the Boston 311 data and use them to predict the outcome of cases from the last 30 days

##Install the package from github using pip

In [1]:
#This library is only needed for the Cox Regression Model, which is not included in this tutorial
#! pip install lifelines

In [2]:
#pwd()

In [3]:
#! pip install ../

##Import the Boston311Model class

In [1]:
! pip show boston311

Name: boston311
Version: 0.1.0
Summary: A package for training machine learning models on Boston 311 data
Home-page: https://github.com/mindfulcoder49/Boston_311
Author: Alex Alcivar
Author-email: alex.g.alcivar49@gmail.com
License: UNKNOWN
Location: /home/briarmoss/.local/lib/python3.10/site-packages
Requires: matplotlib, numpy, pandas, scikit-learn, tensorflow
Required-by: 


In [2]:
from boston311 import Boston311LogReg, Boston311EventDecTree, Boston311SurvDecTree

2023-09-08 18:06:37.937782: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-08 18:06:37.972173: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-08 18:06:37.972686: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-08 18:06:38.442564: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


##Get latest file URLS and Current Date Ranges

I will program several scenarios, eg:

```
linear_tree_model = Boston311SurvDecTree.Boston311SurvDecTree(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type','queue'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0,
                                      'survivalTimeFill':'2023-05-22'},
                            files_dict=latest_URLS)
```

and then i will pick a prediction range to provide predictions for open cases in that range, and every day update the case information, retrain the models and provide new predictions. The front end display will show information about the models, and their predictions. the scenarios and how they were chosen, and the set of cases we are doing predictions for. The predictions will only be recorded in the database if they are made by a model when the case was open. The model id will go back to being a simple integer and not a date because the models will be retrained every day.

In [6]:
logregmodel = Boston311LogReg()
logregmodel.load('logregproperties.json', 'logreg.h5')

In [7]:
from datetime import datetime, timedelta
now = datetime.now()
thirty_days = timedelta(days=30)
thirty_days_ago = now - thirty_days
today_datestring = now.strftime("%Y-%m-%d")
thirty_days_ago_datestring = thirty_days_ago.strftime("%Y-%m-%d")

print(today_datestring, thirty_days_ago_datestring)

2023-09-07 2023-08-08


In [8]:
logregmodel.predict_date_range = {'start': '2023-08-01', 'end': today_datestring}

In [9]:
logregmodel_prediction = logregmodel.predict()

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:251: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Files with different number of columns from File 0:  []
Files with same number of columns as File 0:  [0]
Files with different column order from File 0:  []
Files with same column order as File 0:  [0]
918/918 [==============================] - 4s 4ms/step


In [11]:
import pandas as pd

ml_model_name = 'logregmodel'
ml_model_type = 'logistic regression'
# set ml_model_id to 1 and then today_datestring as an integer, to make a string of digits
ml_model_id = '1' + today_datestring.replace('-','')

#create a new dataframe with new columns, ml_model_name, ml_model_type,ml_model_id
ml_model_df = pd.DataFrame({'ml_model_name': [ml_model_name], 
                                    'ml_model_type': [ml_model_type],
                                    'id': [ml_model_id],
                                    'ml_model_date': [today_datestring]})




In [12]:
#get new dataframe with just the event_prediction column from the logregmodel_prediction dataframe
logregmodel_prediction_event = logregmodel_prediction[['event_prediction','case_enquiry_id']].copy()
logregmodel_prediction_event['ml_model_id'] = ml_model_id
#add today's date to the dataframe
logregmodel_prediction_event['prediction_date'] = today_datestring
#rename the event_prediction column to prediction
logregmodel_prediction_event.rename(columns={'event_prediction': 'prediction'}, inplace=True)


In [13]:
#remove logregmodel_prediction event_prediction column
logregmodel_cases = logregmodel_prediction.drop('event_prediction', axis=1).copy()

In [14]:
#remove geom column in logregmodel_cases
logregmodel_cases = logregmodel_cases.drop('geom_4326', axis=1).copy()

##Save the prediction data

In [16]:
logregmodel_cases.to_csv('311.csv', index=False)


In [17]:

logregmodel_prediction_event.to_csv('311_predictions.csv', index=False)

In [18]:

ml_model_df.to_csv('311_ml_models.csv', index=False)